<a href="https://colab.research.google.com/github/VishnunandP/3D-CAD-Symmetry-Analyzer/blob/main/3DSYM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge pythonocc-core -y

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pythonocc-core


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohappyeyeballs-2.6.1     |     pyhd8ed1ab_0          19 KB  conda-forge
    aiohttp-3.11.18            |  py311h2dc5d0c_0         905 KB  conda-forge
    aiosignal-1.3.2            |     pyhd8ed1ab_0          13 KB  conda-forge
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attrs-25.

In [ ]:

import math
from google.colab import files
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone #status, flag
from OCC.Core.TopExp import TopExp_Explorer #iterates over faces
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.TopoDS import topods #current component converter
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties #props like area, inertia
from OCC.Core.GProp import GProp_GProps #stores geo props ^
from OCC.Core.BRep import BRep_Tool #low level geo data (struct, curves)
from OCC.Core.GeomLProp import GeomLProp_SLProps #normals, curvature locally
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface #adapts topods face to geo
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Core.gp import gp_Pln, gp_Ax3, gp_Dir, gp_Trsf
from scipy.spatial import KDTree #spatial querying
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_MakeFace,
    BRepBuilderAPI_Transform
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakePrism
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs #as is
import numpy as np

uploaded = files.upload()
step_filename = list(uploaded.keys())[0]


def load_step_file(path):
    reader = STEPControl_Reader()
    status = reader.ReadFile(path)
    if status != IFSelect_RetDone:
        raise ValueError("STEP file could not be read.")
    reader.TransferRoots()
    return reader.OneShape()


def extract_faces(shape):
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    faces = []
    while explorer.More():
        faces.append(topods.Face(explorer.Current()))
        explorer.Next()
    return faces

def get_face_area(face):
    props = GProp_GProps()
    brepgprop_SurfaceProperties(face, props)
    return props.Mass()


def get_face_normal(face):
    surf_adapt = BRepAdaptor_Surface(face)
    u_min, u_max = surf_adapt.FirstUParameter(), surf_adapt.LastUParameter()
    v_min, v_max = surf_adapt.FirstVParameter(), surf_adapt.LastVParameter()
    u_mid = 0.5 * (u_min + u_max)
    v_mid = 0.5 * (v_min + v_max)

    geom_surface = BRep_Tool.Surface(face)
    prop = GeomLProp_SLProps(geom_surface, u_mid, v_mid, 1, 1e-6)

    if not prop.IsNormalDefined():
        return gp_Vec(0, 0, 0)

    normal = prop.Normal()
    return normal

def angle_between(v1, v2):
    dot = v1.Dot(v2)
    #v1 v2 unit vs
    cos_theta = max(min(dot, 1), -1)
    return math.degrees(math.acos(cos_theta))

def group_similar_faces(shape, area_tol=1e-3, angle_tol_deg=5.0):
    faces = extract_faces(shape)
    groups = []

    for face in faces:
        area = get_face_area(face)
        normal = get_face_normal(face)
        matched = False

        for group in groups:
            ref_area, ref_normal, _ = group[0]
            if abs(ref_area - area) < area_tol and angle_between(ref_normal, normal) < angle_tol_deg:
                group.append((area, normal, face))
                matched = True
                break

        if not matched:
            groups.append([(area, normal, face)])

    return groups


shape = load_step_file(step_filename)
groups = group_similar_faces(shape)


print(f"\nFound {len([g for g in groups if len(g) >= 2])} groups with 2 or more similar faces.")
for i, group in enumerate(groups, 1):
    if len(group) >= 2:
        print(f"\nGroup {i} has {len(group)} face(s):")
        for j, (area, normal, _) in enumerate(group, 1):
            print(f"  Face {j}: Area={area:.4f}, Normal=({normal.X():.2f}, {normal.Y():.2f}, {normal.Z():.2f})")


Saving JAW_SLIDING.step to JAW_SLIDING.step

Found 19 groups with 2 or more similar faces.

Group 3 has 2 face(s):
  Face 1: Area=131.6933, Normal=(0.00, -0.00, 1.00)
  Face 2: Area=131.6933, Normal=(0.00, -0.00, 1.00)

Group 5 has 2 face(s):
  Face 1: Area=56.1994, Normal=(0.00, -0.57, -0.82)
  Face 2: Area=56.1994, Normal=(0.00, -0.57, -0.82)

Group 10 has 2 face(s):
  Face 1: Area=589.6163, Normal=(1.00, 0.00, 0.00)
  Face 2: Area=589.6163, Normal=(1.00, 0.00, 0.00)

Group 11 has 2 face(s):
  Face 1: Area=151.9713, Normal=(-1.00, 0.00, 0.00)
  Face 2: Area=151.9713, Normal=(-1.00, 0.00, 0.00)

Group 14 has 2 face(s):
  Face 1: Area=307.9631, Normal=(-1.00, 0.00, 0.00)
  Face 2: Area=307.9631, Normal=(-1.00, 0.00, 0.00)

Group 20 has 2 face(s):
  Face 1: Area=104.9934, Normal=(1.00, 0.00, 0.00)
  Face 2: Area=104.9934, Normal=(1.00, 0.00, 0.00)

Group 25 has 2 face(s):
  Face 1: Area=135.6665, Normal=(-0.00, 0.00, -1.00)
  Face 2: Area=135.6665, Normal=(-0.00, 0.00, -1.00)

Group 26 

<ipython-input-18-de6d7e3ed47d>:51: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-18-de6d7e3ed47d>:51: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-18-de6d7e3ed47d>:51: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-18-de6d7e3ed47d>:51: DeprecationWarning: Call to deprecated function brepgprop_SurfacePrope

In [ ]:
def get_face_centroid(face):
    props = GProp_GProps()
    brepgprop_SurfaceProperties(face, props)
    pnt = props.CentreOfMass()
    return gp_Pnt(pnt.X(), pnt.Y(), pnt.Z())


In [ ]:
def get_symmetry_plane_from_face_pair(face1, face2):
    p1 = get_face_centroid(face1)
    p2 = get_face_centroid(face2)

    #midpoint of centroids
    mid_x = (p1.X() + p2.X()) / 2
    mid_y = (p1.Y() + p2.Y()) / 2
    mid_z = (p1.Z() + p2.Z()) / 2
    midpoint = gp_Pnt(mid_x, mid_y, mid_z)

    #direction vector from face1 to face2
    dir_vec = gp_Vec(p1, p2)
    if dir_vec.Magnitude() == 0:
        return None  #means faces coincident

    #normal to plane
    normal_dir = gp_Dir(dir_vec)

    #ymm plane constr
    symmetry_plane = gp_Pln(midpoint, normal_dir)
    return symmetry_plane


In [ ]:
symmetry_planes = []

for group in groups:
    if len(group) == 2:
        face1 = group[0][2]
        face2 = group[1][2]
        plane = get_symmetry_plane_from_face_pair(face1, face2)
        if plane:
            symmetry_planes.append(plane)


<ipython-input-19-dea4cd374e42>:3: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-19-dea4cd374e42>:3: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-19-dea4cd374e42>:3: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-19-dea4cd374e42>:3: DeprecationWarning: Call to deprecated function brepgprop_SurfacePropertie

In [ ]:
for i, plane in enumerate(symmetry_planes, 1):
    origin = plane.Location()
    normal = plane.Axis().Direction()
    print(f"Plane {i}: Origin=({origin.X():.2f}, {origin.Y():.2f}, {origin.Z():.2f}), "
          f"Normal=({normal.X():.2f}, {normal.Y():.2f}, {normal.Z():.2f})")


Plane 1: Origin=(-0.00, 51.44, 80.99), Normal=(-1.00, 0.00, 0.00)
Plane 2: Origin=(0.00, 51.69, 101.20), Normal=(-1.00, 0.00, 0.00)
Plane 3: Origin=(0.00, 45.56, 86.80), Normal=(-1.00, 0.00, 0.00)
Plane 4: Origin=(0.00, 51.16, 91.10), Normal=(1.00, 0.00, 0.00)
Plane 5: Origin=(0.00, 45.31, 62.73), Normal=(-1.00, 0.00, -0.00)
Plane 6: Origin=(0.00, 51.44, 88.40), Normal=(1.00, -0.00, 0.00)
Plane 7: Origin=(0.00, 38.64, 90.20), Normal=(1.00, 0.00, 0.00)
Plane 8: Origin=(0.00, 41.34, 90.20), Normal=(1.00, 0.00, 0.00)
Plane 9: Origin=(0.00, 43.34, 90.20), Normal=(1.00, 0.00, 0.00)
Plane 10: Origin=(-3.02, 57.01, 96.81), Normal=(1.00, 0.00, 0.00)
Plane 11: Origin=(3.02, 57.01, 96.81), Normal=(-1.00, -0.00, -0.00)
Plane 12: Origin=(0.00, 57.30, 96.97), Normal=(1.00, 0.00, 0.00)
Plane 13: Origin=(-0.00, 46.07, 105.02), Normal=(1.00, 0.00, 0.00)
Plane 14: Origin=(-8.75, 57.06, 81.13), Normal=(1.00, 0.00, 0.00)
Plane 15: Origin=(8.75, 57.06, 81.13), Normal=(-1.00, 0.00, 0.00)
Plane 16: Origin=(

In [ ]:
#get face center as np.array
def get_face_center(face):
    props = GProp_GProps()
    brepgprop_SurfaceProperties(face, props)
    center = props.CentreOfMass()
    return np.array([center.X(), center.Y(), center.Z()])

#reflect point across plane
def mirror_point_across_plane(point, plane: gp_Pln):
    plane_origin = np.array([plane.Location().X(), plane.Location().Y(), plane.Location().Z()])
    normal = np.array([plane.Axis().Direction().X(), plane.Axis().Direction().Y(), plane.Axis().Direction().Z()])
    to_point = point - plane_origin
    distance = np.dot(to_point, normal)
    mirrored = point - 2 * distance * normal
    return mirrored

def find_global_mirror_plane(symmetry_planes, groups, tol=1e-2, min_match_ratio=0.4):
    #all face centers in single list for eval
    all_faces = [face for group in groups.values() for face in group]
    centers = [get_face_center(f) for f in all_faces]
    kdtree = KDTree(centers) #spatial querying

    best_plane = None
    best_match_count = 0

    for plane in symmetry_planes:
        matched = 0
        for center in centers:
            mirrored = mirror_point_across_plane(center, plane)
            dist, idx = kdtree.query(mirrored)
            if dist < tol:
                matched += 1

        if matched > best_match_count:
            best_match_count = matched
            best_plane = plane

    match_ratio = best_match_count / len(centers)
    if match_ratio >= min_match_ratio:
        print(f"Best match ratio: {match_ratio:.2f}")
        return best_plane
    else:
        print("No global symmetry plane found with sufficient matching")
        return None

In [ ]:
group_dict = {i: [face_info[2] for face_info in group] for i, group in enumerate(groups)}


In [ ]:
best_plane = find_global_mirror_plane(symmetry_planes, group_dict)


Best match ratio: 1.00


<ipython-input-23-97fee81e069e>:4: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-23-97fee81e069e>:4: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-23-97fee81e069e>:4: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)
<ipython-input-23-97fee81e069e>:4: DeprecationWarning: Call to deprecated function brepgprop_SurfacePropertie

In [ ]:
if best_plane:
    origin = best_plane.Location()
    normal = best_plane.Axis().Direction()
    print(f"Mirror Plane origin: ({origin.X():.2f}, {origin.Y():.2f}, {origin.Z():.2f})")
    print(f"Mirror Plane normal: ({normal.X():.2f}, {normal.Y():.2f}, {normal.Z():.2f})")


Best symmetry plane - Mirror Plane origin: (-0.00, 51.44, 80.99)
Best symmetry plane - Mirror Plane normal: (-1.00, 0.00, 0.00)


In [ ]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add

def visualize_centered_mirror_plane(shape, plane, size_factor=1.5, thickness=3.0):

    # bounding box
    bbox = Bnd_Box()
    brepbndlib_Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()

    # width needed for visn
    width = max(xmax - xmin, ymax - ymin, zmax - zmin) * size_factor

    # mirror planes origin and normal
    origin = plane.Location()
    normal = plane.Axis().Direction()

    # toprint
    print(f"Visualizing Plane -> Origin: ({origin.X():.2f}, {origin.Y():.2f}, {origin.Z():.2f}), "
          f"Normal: ({normal.X():.2f}, {normal.Y():.2f}, {normal.Z():.2f})")

    # constructing plane
    face = BRepBuilderAPI_MakeFace(plane, -width / 2, width / 2, -width / 2, width / 2).Face()

    # thickness of plane
    extrusion_vec = gp_Vec(normal).Scaled(thickness)
    plane_solid = BRepPrimAPI_MakePrism(face, extrusion_vec).Shape()

    # fusion
    plane_fused = BRepAlgoAPI_Fuse(shape, plane_solid).Shape()

    # export
    out_file = "visualized_mirror_plane.step"
    step_writer = STEPControl_Writer()
    status = step_writer.Transfer(plane_fused, STEPControl_AsIs)

    if status == IFSelect_RetDone:
        step_writer.Write(out_file)
        files.download(out_file)
        print(f"\n Visualization is '{out_file}'.")
    else:
        raise RuntimeError("STEP export failed.")


In [ ]:
if best_plane:
    visualize_centered_mirror_plane(shape, best_plane)
else:
    print("No valid mirror plane found.")


Visualizing Plane -> Origin: (-0.00, 51.44, 80.99), Normal: (-1.00, 0.00, 0.00)


<ipython-input-27-f25e68df5583>:15: DeprecationWarning: Call to deprecated function brepbndlib_Add since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepbndlib.Add
  brepbndlib_Add(shape, bbox)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Visualization exported as 'visualized_mirror_plane.step'.
